<a href="https://colab.research.google.com/github/alisonnnnn88/programming_language/blob/main/HW5_%E9%A3%AF%E5%BA%97%E6%9F%A5%E8%A9%A2%E5%9C%B0%E5%9C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gradio as gr
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd
import requests
from geopy.geocoders import Nominatim
import random
import folium

# Google 認證
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheets URL (請替換為您的 Google Sheets URL)
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1h75idXVO6GMosL5bPUF-G76MqxOziHu1tJieRtpdkP4/edit?usp=sharing"
sh = gc.open_by_url(SPREADSHEET_URL)

# 設定工作表
worksheet = sh.worksheet("飯店資訊")

# ==========================
# 初始化地理編碼器
# ==========================
geolocator = Nominatim(user_agent="hotel_search_app")

def get_coordinates(location_name):
    try:
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except:
        return None

# ==========================
# 取得完整地址函數
# ==========================
def get_full_address(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='zh-TW', timeout=10)
        if location:
            return location.address
        return "無地址"
    except:
        return "無地址"

# ==========================
# 抓附近飯店資料
# ==========================
def fetch_hotels(lat, lon, radius=2000):
    query = f"""
    [out:json];
    node
      [tourism~"hotel|motel|guest_house"]
      (around:{radius},{lat},{lon});
    out;
    """
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data={'data': query})
    data = response.json()

    hotels = []
    for el in data.get("elements", []):
        tags = el.get("tags", {})
        hotel_lat = el["lat"]
        hotel_lon = el["lon"]
        full_address = get_full_address(hotel_lat, hotel_lon)

        hotels.append({
            "飯店名稱": tags.get("name", "未知飯店"),
            "評價": round(random.uniform(3.0,5.0),1),
            "價位": random.randint(2000,8000),
            "地址": full_address,
            "含早餐": random.choice([True, False]),
            "寵物友善": random.choice([True, False]),
            "有停車場": random.choice([True, False]),
            "緯度": hotel_lat,
            "經度": hotel_lon
        })

    # 若無資料，補模擬飯店
    if not hotels:
        for i in range(5):
            lat_jitter = lat + random.uniform(-0.02,0.02)
            lon_jitter = lon + random.uniform(-0.02,0.02)
            full_address = get_full_address(lat_jitter, lon_jitter)
            hotels.append({
                "飯店名稱": f"模擬飯店{i+1}",
                "評價": round(random.uniform(3.0,5.0),1),
                "價位": random.randint(2000,8000),
                "地址": full_address,
                "含早餐": random.choice([True, False]),
                "寵物友善": random.choice([True, False]),
                "有停車場": random.choice([True, False]),
                "緯度": lat_jitter,
                "經度": lon_jitter
            })

    df = pd.DataFrame(hotels)

    # ✅ 轉成「是 / 否」呈現
    for col in ["含早餐", "寵物友善", "有停車場"]:
        df[col] = df[col].map({True: "是", False: "否"})

    return df

# ==========================
# 篩選 & AI 推薦 & 地圖生成
# ==========================
def search_and_recommend_and_write_to_sheets(location, min_rating, max_price, checkin, checkout,
                                             breakfast, pet, parking, radius, sort_by, num_guests, num_rooms):
    coords = get_coordinates(location)
    if not coords:
        return None, "<p>無法取得地點座標</p>", "請輸入正確地名", None
    lat, lon = coords

    df_hotels = fetch_hotels(lat, lon, radius)
    if df_hotels.empty:
        return None, "<p>此地區找不到飯店</p>", "（無建議）", None

    # 篩選條件
    filtered = df_hotels[
        (df_hotels['評價'] >= min_rating) &
        (df_hotels['價位'] <= max_price)
    ]
    if breakfast:
        filtered = filtered[filtered['含早餐'] == "是"]
    if pet:
        filtered = filtered[filtered['寵物友善'] == "是"]
    if parking:
        filtered = filtered[filtered['有停車場'] == "是"]

    if filtered.empty:
        filtered = df_hotels.sample(min(5, len(df_hotels)))

    # 排序方式
    if sort_by == "評價高→低":
        filtered = filtered.sort_values(by="評價", ascending=False)
    elif sort_by == "價位低→高":
        filtered = filtered.sort_values(by="價位", ascending=True)
    else:  # AI推薦
        filtered["AI分數"] = filtered["評價"] * 0.6 + (1 - filtered["價位"]/filtered["價位"].max()) * 0.4
        filtered = filtered.sort_values(by="AI分數", ascending=False)

    # 只保留需要寫入表格的列
    filtered_to_write = filtered[['飯店名稱', '評價', '價位', '地址', '含早餐', '寵物友善', '有停車場']]

    # 將資料寫入 Google Sheets
    for _, row in filtered_to_write.iterrows():
        worksheet.append_row(row.tolist())

    # AI推薦條列
    ai_summary_lines = []
    for i, row in filtered.head(3).iterrows():
        line = f"🏨 {row['飯店名稱']}\n⭐ 評價: {row['評價']}\n💰 價位: {row['價位']}\n🍳 含早餐: {row['含早餐']}\n🐶 寵物友善: {row['寵物友善']}\n🚗 有停車場: {row['有停車場']}\n入住: {checkin} 退房: {checkout}"
        ai_summary_lines.append(line)
    ai_summary = f"入住：{checkin}，退房：{checkout}，人數：{num_guests}，房間數：{num_rooms}\n" + "\n".join(ai_summary_lines)

    # 地圖顯示
    def get_color(rating):
        if rating >= 4.5:
            return "green"
        elif rating >= 4.0:
            return "blue"
        elif rating >= 3.5:
            return "orange"
        else:
            return "red"

    m = folium.Map(location=[lat, lon], zoom_start=13)
    folium.Circle(
        location=[lat, lon],
        radius=radius,
        color="blue",
        fill=True,
        fill_opacity=0.05,
        tooltip=f"搜尋範圍 {radius/1000:.1f} 公里"
    ).add_to(m)

    # 標註飯店位置
    for _, row in filtered.iterrows():
       # 判斷是否為推薦飯店
        if row["飯店名稱"] in filtered.head(3)["飯店名稱"].values:
            icon_color = "purple"  # 推薦飯店使用粉紅色
            icon = "star"  # 星形圖標
        else:
            icon_color = get_color(row['評價']) # 根據評價設定顏色
            icon = "home"  # 其他飯店使用家形圖標

        folium.Marker(
            [row["緯度"], row["經度"]],
            tooltip=f"{row['飯店名稱']}｜⭐{row['評價']}｜💰{row['價位']}｜<br>{row['地址']}",
            icon=folium.Icon(color=icon_color, icon=icon, prefix="fa")
        ).add_to(m)

    return filtered_to_write, ai_summary, m._repr_html_()

# Gradio 介面設定
TAIWAN_CITIES = [
    "台北市", "新北市", "基隆市", "桃園市", "新竹市", "新竹縣",
    "苗栗縣", "台中市", "彰化縣", "南投縣", "雲林縣", "嘉義市", "嘉義縣",
    "台南市", "高雄市", "屏東縣", "宜蘭縣", "花蓮縣", "台東縣",
    "澎湖縣", "金門縣", "連江縣"
]

with gr.Blocks(title="飯店搜尋與 AI 推薦") as demo:
    gr.Markdown("## 🏨 台灣飯店搜尋與 AI 推薦")

    with gr.Row():
        city_input = gr.Dropdown(label="目的地", choices=TAIWAN_CITIES, value="台北市")
        rating_slider = gr.Slider(0, 5, value=3.5, step=0.1, label="最低評價")
        price_slider = gr.Slider(1000, 20000, value=8000, step=500, label="最高價位")
    with gr.Row():
        breakfast = gr.Checkbox(label="只顯示含早餐", value=False)
        pet = gr.Checkbox(label="只顯示寵物友善", value=False)
        parking = gr.Checkbox(label="只顯示有停車場", value=False)
    with gr.Row():
        radius_slider = gr.Slider(500, 5000, value=2000, step=500, label="搜尋半徑 (公尺)")
        sort_by = gr.Radio(["評價高→低", "價位低→高", "AI推薦"], label="排序方式", value="AI推薦")
    with gr.Row():
        checkin_input = gr.Textbox(label="入住日期 (YYYY-MM-DD)")
        checkout_input = gr.Textbox(label="退房日期 (YYYY-MM-DD)")
        num_guests = gr.Slider(1, 10, value=2, step=1, label="入住人數")
        num_rooms = gr.Slider(1, 5, value=1, step=1, label="房間數")

    search_btn = gr.Button("🔎 搜尋飯店")

    map_output = gr.HTML(label="地圖")
    table_output = gr.DataFrame(label="搜尋結果", interactive=False)
    ai_output = gr.Textbox(label="AI推薦飯店", lines=10, max_lines=20, interactive=False)

    # 按下搜尋按鈕時觸發
    search_btn.click(
        fn=search_and_recommend_and_write_to_sheets,
        inputs=[city_input, rating_slider, price_slider, checkin_input, checkout_input,
                breakfast, pet, parking, radius_slider, sort_by, num_guests, num_rooms],
        outputs=[table_output, ai_output, map_output]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6759a88d7802d235c5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
